In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator

payload=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
#print(payload)
boom=pd.DataFrame()

class algo:
    start="2015-01-01"
    end="2021-12-01"
    cutoff='2021-01-01'
    interval='1mo'
    thr1=1
    thr2=99
    
    def __init__(self,stock):
        self.stock=stock
       
    def getdata(self):
        df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())
        df['dlr_day']=np.log(df.Close/df.Open)
        df['stock']=self.stock
        
        return(df)


    def indicators(self):
        df=self.stock
        df['dlr']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        df=df.dropna()
        
        return(df)

    def percentiles(self):
        df=self.stock
        df=(df[df.index<algo.cutoff])
        p1=np.percentile(df.dlr,algo.thr1)
        p2=np.percentile(df.dlr,algo.thr2)
        
        return(p1,p2)
    
    def trade(self,percs):
        df=self.stock
        df=(df[df.index>=algo.cutoff])
        df['trade_pass']=np.select([(df.dlr<percs[0]),(df.dlr>percs[1])],[1,-1],default=0)
        long=df[df['trade_pass']==1]
        short=df[df['trade_pass']==-1]

        return(df,long,short)
    
    def parse(i):
        rawdata=algo(payload[i]).getdata()
        ta=algo(rawdata).indicators()
        percs=algo(ta).percentiles()
        alll,long,short=algo(ta).trade(percs)
        
        return(payload[i],(np.sum(long.dlr_day)),(np.sum(alll.dlr_day)),(np.nanmean(long.dlr_day)),(np.nanmean(alll.dlr_day)))
        
temp=np.array([])
k=0
l1=0
l2=len(payload)
for i in range (l1,l2):
    try:
        temp=np.append(temp,(algo.parse(i)))
        rawdata=algo(payload[i]).getdata()
        ta=algo(rawdata).indicators()
        percs=algo(ta).percentiles()
        alll,long,short=algo(ta).trade(percs)
        boom=pd.concat([boom,long])
    except Exception as e:
        print(e)
        k=k+1
        
        
#print(np.nan_to_num(temp.reshape(i+1-k-l1,5),nan=0))
dfout=pd.DataFrame(temp.reshape(i+1-k-l1,5))
dfout.loc[dfout[3] == 'nan', 3] = 0
print(dfout)
dfout.to_csv('2021_bt.csv')
boom.to_csv('boomer_trades.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
print(boom)


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-10-01  176.250000  183.820007  173.779999  178.679993  177.237610   
2021-10-01  118.199997  129.399994  114.400002  128.889999  128.394272   
2021-10-01  108.250000  114.919998  106.860001  114.669998  113.296448   
2021-02-01  246.100006  261.609985  243.970001  250.899994  248.683517   
2021-10-01  323.369995  359.619995  316.010010  358.790009  357.727966   
2021-08-01   84.029999   86.389999   77.809998   82.370003   82.370003   
2021-02-01  462.279999  506.510010  450.119995  459.670013  459.670013   
2021-10-01  577.520020  659.190002  552.140015  650.359985  650.359985   
2021-06-01   25.570000   27.000000   24.629999   26.070000   25.752127   
2021-08-01   23.920000   25.480000   23.450001   23.870001   23.726572   

                 Volume   dlr_day stock        dlr  trade_pass  
Date                                          